### Scripts to construct a water balance from LIS ouptut, for GRACE comparison and forward modeling

In [1]:
# Import the necessary libraries

%matplotlib inline
import os
import sys
import datetime

import xarray as xr

import pandas as pd

from dask.diagnostics import ProgressBar

import seaborn as sb

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

from himatpy.LIS import utils as LISutils

import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

In [2]:
# Open all files into a single xarray dataset
ds = LISutils.get_xr_dataset('/att/pubrepo/hma_data/products/LIS-new/', fname=None, multiple_nc=True)

In [ ]:
# Searilizing to new NetCDF files to select only the variables we need
# had to do this in yearly blocks due to errors in handling that many daily files (an issue with Dask I think)

for yr in range(2003,2014):
    startDate = datetime.datetime(yr,1,1)
    endDate = datetime.datetime(yr,12,31)
    ds[['Qsm_tavg','Rainf_tavg','Qs_tavg','Snowf_tavg','Qsb_tavg','Evap_tavg','TWS_tavg']].sel \
    (time = slice('{:{dtfmt}}'.format(startDate, dtfmt='%Y%m%d'), \
                  '{:{dtfmt}}'.format(endDate, dtfmt='%Y%m%d'))).to_netcdf \
    ('/att/nobackup/aarendt/LIS/LIS_{}.nc'.format(yr))
    print(yr)

In [ ]:
## FIRST, searlize this to a single netcdf, retaining the daily resolution but keeping only these 
## variables and doing the unit adjustments:

# The variables we want are:

{'Qsm_tavg','Rainf_tavg','Qs_tavg','Snowf_tavg','Qsb_tavg','Evap_tavg','TWS_tavg'}

# The associated names are:
{'Snowmelt','Rainfall','SurfaceRunoff','Snowfall','SubsfcRunoff','Evapotranspiration','TWS'}

# multiplication factor for units

{864000,864000,864000,864000,864000,864000,0}

# attributes for the first 6 variables:

{'Daily <snowmelt> in units of mm we', '....'}

# attributes for the TWS:

{'Daily change in water storage'}

In [ ]:
## NEXT calculate monthly sums, e.g.:

with ProgressBar():
    TWS_monthly = ds['TWS_tavg'].resample('MS', 'time', how = 'sum')

# attributes for the first 6 variables:

{'Cumulative monthly <snowmelt> in units of mm we','Cumulative monthly ....'}

# attributes for TWS:

{'Change in monthly water storage'}
       
# then concatenate all into a single netcdf and serialize to /att/nobackup/aarendt    
    
        